Data Summarization

Load and Preview the Data

In [ ]:
import pandas as pd
# Load the dataset
df = pd.read_csv(r"C:\10 Kifia Tasks\Week-3\Insurance-Risk-Analytics\Data\Data_insurance.txt", delimiter='|')
df = df.sample(n=2000, random_state=42)

# Preview structure
df.head()
df.shape


Data Structure Overview

In [4]:
# General info
df.info()

# Column-wise data types
df.dtypes.value_counts()

# Check for datetime fields
if 'TransactionMonth' in df.columns:
    df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')

# Detect mixed-type columns
object_cols = df.select_dtypes(include='object')
for col in object_cols.columns:
    print(f"{col}:", df[col].unique()[:5])  # Show sample categories


<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 57024 to 889637
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   UnderwrittenCoverID       2000 non-null   int64  
 1   PolicyID                  2000 non-null   int64  
 2   TransactionMonth          2000 non-null   object 
 3   IsVATRegistered           2000 non-null   bool   
 4   Citizenship               2000 non-null   object 
 5   LegalType                 2000 non-null   object 
 6   Title                     2000 non-null   object 
 7   Language                  2000 non-null   object 
 8   Bank                      1696 non-null   object 
 9   AccountType               1914 non-null   object 
 10  MaritalStatus             1986 non-null   object 
 11  Gender                    1985 non-null   object 
 12  Country                   2000 non-null   object 
 13  Province                  2000 non-null   object 
 14  PostalC

Descriptive Statistics for Numerical Features

In [5]:
# Select key numeric columns
numeric_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 'CalculatedPremiumPerTerm']
numeric_df = df[numeric_cols]

# Summary stats
summary_stats = numeric_df.describe().T

# Add coefficient of variation
summary_stats['CoV'] = (numeric_df.std() / numeric_df.mean()).round(2)

summary_stats


,count,mean,std,min,25%,50%,75%,max,CoV
TotalPremium,2000.0,58.465442,146.777852,0.0000,0.000000,2.039781,21.929825,1046.989123,2.51
TotalClaims,2000.0,173.568855,6277.091157,0.0000,0.000000,0.000000,0.000000,276248.859649,36.16
CustomValueEstimate,438.0,207730.011416,102502.263843,23100.0000,116300.000000,216350.000000,280000.000000,654000.000000,0.49
CalculatedPremiumPerTerm,2000.0,112.944281,208.489163,0.0002,3.257825,8.175150,90.000000,1193.567600,1.85


Missing Value Check

In [6]:
missing_data = df.isnull().sum()
missing_ratio = (missing_data / len(df)).sort_values(ascending=False)

# Show top 10 missing columns
missing_ratio.head(10)


NumberOfVehiclesInFleet    1.0000
CrossBorder                0.9990
CustomValueEstimate        0.7810
Rebuilt                    0.6655
Converted                  0.6655
WrittenOff                 0.6655
Bank                       0.1520
NewVehicle                 0.1435
AccountType                0.0430
Gender                     0.0075
dtype: float64

Univariate Analysis

Plot Histograms for Numerical Variables

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Ensure plots folder exists
os.makedirs("plots", exist_ok=True)

numeric_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 'CalculatedPremiumPerTerm']

for col in numeric_cols:
    plt.figure(figsize=(7, 4))
    sns.histplot(data=df, x=col, bins=30, kde=True, color='steelblue')
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig(f"plots/hist_{col.lower()}.png", dpi=300)
    plt.close()


Bar Charts for Categorical Variables

In [ ]:
categorical_cols = ['Province', 'VehicleType', 'Gender']

for col in categorical_cols:
    plt.figure(figsize=(8, 4))
    order = df[col].value_counts().index
    sns.countplot(data=df, x=col, order=order, palette='viridis')
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"plots/bar_{col.lower()}.png", dpi=300)
    plt.close()


Bivariate/ Multivariate Analysis

Group Data by Month & ZipCode

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv(r"C:\10 Kifia Tasks\Week-3\Insurance-Risk-Analytics\Data\Data_insurance.txt", delimiter='|')
df = df.sample(n=2000, random_state=42)

 # Replace with actual path
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])

# Group and compute monthly changes
monthly = df.groupby(['PostalCode', 'TransactionMonth'])[['TotalPremium', 'TotalClaims']].sum().reset_index()
monthly.sort_values(['PostalCode', 'TransactionMonth'], inplace=True)
monthly[['PremiumDelta', 'ClaimsDelta']] = monthly.groupby('PostalCode')[['TotalPremium', 'TotalClaims']].diff()
monthly_clean = monthly.dropna(subset=['PremiumDelta', 'ClaimsDelta'])

# Compute and visualize correlation
correlation_matrix = monthly_clean[['PremiumDelta', 'ClaimsDelta']].corr()
plt.figure(figsize=(6,4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix: ΔPremium vs ΔClaims')
plt.tight_layout()
plt.savefig("plots/correlation_heatmap.png", dpi=300)


Scatterplot Visualizations

In [ ]:
# Color-coded scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=monthly_clean,
    x='PremiumDelta',
    y='ClaimsDelta',
    hue='PostalCode',
    alpha=0.6,
    palette='tab20'
)
plt.title('Monthly ΔPremium vs ΔClaims Colored by ZipCode')
plt.xlabel('Premium Change')
plt.ylabel('Claims Change')
plt.legend(title='ZipCode', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("plots/monthly_delta_scatter_zipcode.png")
plt.close()


Data comparison over Geography

In [46]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# 🔹 1. Average Monthly Premium by Province
premium_trend = df.groupby(['Province', 'TransactionMonth'])['TotalPremium'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(data=premium_trend, x='TransactionMonth', y='TotalPremium', hue='Province', marker='o')
plt.title('Average Monthly Premium by Province')
plt.xlabel('Month')
plt.ylabel('Average Premium')
plt.tight_layout()
plt.savefig('plots/trend_avg_premium_province.png')
plt.close()

# 🔹 2. CoverType Distribution by Province
cover_counts = df.groupby(['Province', 'CoverType']).size().reset_index(name='Count')

plt.figure(figsize=(12, 6))
sns.barplot(data=cover_counts, x='Province', y='Count', hue='CoverType')
plt.title('CoverType Distribution Across Provinces')
plt.xlabel('Province')
plt.ylabel('Number of Policies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/covertype_distribution_by_province.png')
plt.close()

# 🔹 3. Top Auto Makes by Province (Grouped Bar Plot)
make_counts = df.groupby(['Province', 'make']).size().reset_index(name='Count')
top_makes = make_counts.groupby('make')['Count'].sum().nlargest(5).index.tolist()
filtered_make = make_counts[make_counts['make'].isin(top_makes)]

plt.figure(figsize=(12, 6))
sns.barplot(data=filtered_make, x='Province', y='Count', hue='make')
plt.title('Top Auto Makes by Province')
plt.xlabel('Province')
plt.ylabel('Vehicle Frequency')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/top_auto_make_by_province.png')
plt.close()


Outlier Detection

In [47]:
import seaborn as sns
import matplotlib.pyplot as plt
import os


for col in numeric_cols:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x=df[col])
    plt.title(f'Box Plot for {col}')
    plt.xlabel(col)
    plt.tight_layout()
    plt.savefig(f'plots/boxplot_{col}.png')
    plt.close()


Visualizations

Risk Heatmap: Loss Ratio by Province and Vehicle Type

In [48]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

# Compute loss ratio
loss_df = df.groupby(['Province', 'VehicleType'])[['TotalClaims', 'TotalPremium']].sum().reset_index()
loss_df['LossRatio'] = loss_df['TotalClaims'] / loss_df['TotalPremium']

# Pivot for heatmap
loss_pivot = loss_df.pivot(index='Province', columns='VehicleType', values='LossRatio')

plt.figure(figsize=(12, 6))
sns.heatmap(loss_pivot, annot=True, cmap='YlOrRd', fmt=".2f")
plt.title('Loss Ratio Heatmap: Province vs Vehicle Type')
plt.xlabel('Vehicle Type')
plt.ylabel('Province')
plt.tight_layout()
plt.savefig("plots/loss_ratio_heatmap.png")
plt.close()


Vehicle Make vs Claims: Bubble Plot

In [51]:
# Aggregate claim metrics by make
claims_by_make = df.groupby('make').agg(
    TotalClaims=('TotalClaims', 'sum'),
    NumPolicies=('PolicyID', 'nunique')
).reset_index()

# Calculate average claim per policy
claims_by_make['AvgClaimPerPolicy'] = claims_by_make['TotalClaims'] / claims_by_make['NumPolicies']
top_makes = claims_by_make.nlargest(10, 'TotalClaims')

plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=top_makes,
    x='NumPolicies',
    y='AvgClaimPerPolicy',
    size='TotalClaims',
    hue='make',
    sizes=(100, 1000),
    legend=False,
    alpha=0.7
)
plt.title('Claims vs Policy Volume by Vehicle Make')
plt.xlabel('Number of Policies')
plt.ylabel('Average Claim per Policy')
plt.tight_layout()
plt.savefig('plots/bubble_claims_by_make.png')
plt.close()


Province-Based CoverType Distribution

In [52]:
cover_counts = df.groupby(['Province', 'CoverType']).size().reset_index(name='Count')

plt.figure(figsize=(12, 6))
sns.barplot(data=cover_counts, x='Province', y='Count', hue='CoverType')
plt.title('CoverType Preferences Across Provinces')
plt.xlabel('Province')
plt.ylabel('Number of Policies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/viz2_covertype_distribution_province.png')
plt.close()
